# Quantum Computing simulation or Mathematical modeling using "numpy" on Classical Computer

**Author : 2777.kk<br/>Concept developed on : November 20, 2025**

Quantum computing platforms are already been Democratized and available for public consumption globally. Qiskit for instance has already been offering computation on Quantum platform for over 5 years.

True fun begins when you make your hands dirty. Building a real time Quantum Computer is, needless to say, if not impossible (for a layman of course it is), is Billionars Project as you need to be a Multi Billionair and fund multidisciplinary sources(including Physics, Quantum Physics, Engineering streams, Software development) and most importantly "time" to have one 'fault tolerant'(NISQ) Physical Qubits system(chandelier - that you see often commercially on various social  and media platforms).

I hold professional certification in Quantum Computing and having proficiency in Qiskit, Quantum Algorithm and being a post-graduate in Physics, Quantum Physics has always been nostolgic to me. 

On November 20, 2025, I stumbled upon idea what if I can simulate a Quantum computer using<br/>
Data Science tool such as numpy and mimic Qiskit kinda coding?. 

With that amateurish  motivation, I created this small project to demo a naive way of perforing Quantum Computatio on Classical Computer using numpy library.

**Status: Under development as on November 24, 2025**

--------------------------------------------------------------------------------------------------------------------------------

In [8]:
from libs import *

In [9]:
%run libs/Nearest_Kronicker.ipynb

In [10]:
#####-------------------#####

In [11]:
#(1): get state representation
#(2): apply swap operation
#(2): get individual states
#(3): apply reduce kron prod
#(4): perform matrix multiplication with CNOT
#(5): apply nearest_kron_prod
#(6): store the respective kron_prod_out into qubit indices

In [12]:
class QuantumCircuit:
    def __init__(self, num_of_qubits):
        self.n = num_of_qubits
        self._STATE_CACHE_HASH_MAP = dict()
        self._LINE_INDEX_MAP = dict()
        print("Initialize by <instance>.init()")
        
    def init(self):
        self._STATE_CACHE_HASH_MAP = {uuid.uuid4().hex : KET_0 for i in range(0, self.n)}
        self._LINE_INDEX_MAP = {j :hashval for j, hashval in enumerate(self._STATE_CACHE_HASH_MAP.keys())}
            

    
    def state_vector(self):
        """
        Returns Global State of all qubits only
        """
        global_state = [state_info for hashval, state_info in self._STATE_CACHE_HASH_MAP.items()]
        return reduce(kron, global_state)
        
            


    def X(self, on_qubits):
        if isinstance(on_qubits, int):
            hashval = self._LINE_INDEX_MAP[on_qubits]
            curr_state = self._STATE_CACHE_HASH_MAP[hashval] 
            x_operated =matmul(X, curr_state)
            self._STATE_CACHE_HASH_MAP[hashval] =x_operated

        if isinstance(on_qubits, list):
            state_list = list()
            for qubit in on_qubits:
                hashval = self._LINE_INDEX_MAP[qubit]
                curr_state = self._STATE_CACHE_HASH_MAP[hashval]
                self._STATE_CACHE_HASH_MAP[hashval] = matmul(X, curr_state)

    def Y(self, on_qubits):
        if isinstance(on_qubits, int):
            hashval = self._LINE_INDEX_MAP[on_qubits]
            curr_state = self._STATE_CACHE_HASH_MAP[hashval] 
            y_operated =matmul(Y, curr_state)
            self._STATE_CACHE_HASH_MAP[hashval] =y_operated

        if isinstance(on_qubits, list):
            state_list = list()
            for qubit in on_qubits:
                hashval = self._LINE_INDEX_MAP[qubit]
                curr_state = self._STATE_CACHE_HASH_MAP[hashval]
                self._STATE_CACHE_HASH_MAP[hashval] = matmul(Y, curr_state)

    def Z(self, on_qubits):
        if isinstance(on_qubits, int):
            hashval = self._LINE_INDEX_MAP[on_qubits]
            curr_state = self._STATE_CACHE_HASH_MAP[hashval] 
            z_operated =matmul(Z, curr_state)
            self._STATE_CACHE_HASH_MAP[hashval] =z_operated

        if isinstance(on_qubits, list):
            state_list = list()
            for qubit in on_qubits:
                hashval = self._LINE_INDEX_MAP[qubit]
                curr_state = self._STATE_CACHE_HASH_MAP[hashval]
                self._STATE_CACHE_HASH_MAP[hashval] = matmul(Z, curr_state)


    def H(self, on_qubits):
        if isinstance(on_qubits, int):
            hashval = self._LINE_INDEX_MAP[on_qubits]
            curr_state = self._STATE_CACHE_HASH_MAP[hashval] 
            h_operated =matmul(H, curr_state)
            self._STATE_CACHE_HASH_MAP[hashval] =h_operated

        if isinstance(on_qubits, list):
            state_list = list()
            for qubit in on_qubits:
                hashval = self._LINE_INDEX_MAP[qubit]
                curr_state = self._STATE_CACHE_HASH_MAP[hashval]
                self._STATE_CACHE_HASH_MAP[hashval] = matmul(H, curr_state)

    def CX(self, source_qubit, target_qubit):
        """
        CX is a Two qubit Universal Gate. It is also called Non-local gate( or :At a distance"from Einstein's(EPR Paradox based - Spooky action 'at distance').
            It is a Unitary Gate.
            #(1): get state representation
            #(2): apply swap operation
            #(2): get individual states
            #(3): apply reduce kron prod
            #(4): perform matrix multiplication with CNOT
            #(5): apply nearest_kron_prod
            #(6): store the respective kron_prod_out into qubit indices
        """
        qubits_gap = source_qubit - target_qubit
        if source_qubit < self.n and target_qubit < self.n and abs(qubits_gap) >=1:
            if qubits_gap<0:
                #From Source Qubit side
                source_qubit_hash = self._LINE_INDEX_MAP[source_qubit]
                source_qubit_state = self._STATE_CACHE_HASH_MAP[source_qubit_hash]
                #From Target Qubit side
                target_qubit_hash = self._LINE_INDEX_MAP[target_qubit]
                target_qubit_state = self._STATE_CACHE_HASH_MAP[target_qubit_hash]
                cx_current_state = matmul(CNOT, reduce(kron, [source_qubit_state, target_qubit_state]))
                nkp_state = nearest_kron_prod(cx_current_state)
                if nkp_state['ent']:
                    new_hash_key = uuid.uuid4().hex
                    """***Source side ***"""  
                    old_src_hash = self._LINE_INDEX_MAP[source_qubit]
                    self._LINE_INDEX_MAP[source_qubit] = new_hash_key 
                    self._STATE_CACHE_HASH_MAP[new_hash_key] = nkp_state['qubit-pair']
                    del self._STATE_CACHE_HASH_MAP[old_src_hash]  
                    """***Target side ***"""                    
                    old_target_hash = self._LINE_INDEX_MAP[target_qubit]
                    self._LINE_INDEX_MAP[target_qubit] = new_hash_key  
                    self._STATE_CACHE_HASH_MAP[new_hash_key] = nkp_state['qubit-pair']
                    del self._STATE_CACHE_HASH_MAP[old_target_hash]

                if not nkp_state['ent']:
                    src_hashval = self._LINE_INDEX_MAP[source_qubit]
                    self._STATE_CACHE_HASH_MAP[src_hashval]= nkp_state['qubit-pair'][0]
                    tgt_hashval = self._LINE_INDEX_MAP[target_qubit]
                    self._LINE_INDEX_MAP[tgt_hashval] = nkp_state['qubit-pair'][1]

            if qubits_gap>0:
                source_qubit_hash = self._LINE_INDEX_MAP[source_qubit]
                source_qubit_state = self._STATE_CACHE_HASH_MAP[source_qubit_hash]
                target_qubit_hash = self._LINE_INDEX_MAP[target_qubit]
                target_qubit_state = self._STATE_CACHE_HASH_MAP[target_qubit_hash]
                cx_current_state = matmul(INV_CNOT, reduce(kron, [source_qubit_state, target_qubit_state]))
                nkp_state =  nearest_kron_prod(cx_current_state)
                if nkp_state['ent']:
                    new_hash_key = uuid.uuid4().hex
                    """***Source side ***"""  
                    old_src_hash = self._LINE_INDEX_MAP[source_qubit]
                    self._LINE_INDEX_MAP[source_qubit] = new_hash_key 
                    self._STATE_CACHE_HASH_MAP[new_hash_key] = nkp_state['qubit-pair']
                    del self._STATE_CACHE_HASH_MAP[old_src_hash]  
                    """***Target side ***"""                    
                    old_target_hash = self._LINE_INDEX_MAP[target_qubit]
                    self._LINE_INDEX_MAP[target_qubit] = new_hash_key  
                    self._STATE_CACHE_HASH_MAP[new_hash_key] = nkp_state['qubit-pair']
                    del self._STATE_CACHE_HASH_MAP[old_target_hash]

                if not nkp_state['ent']:
                    src_hashval = self._LINE_INDEX_MAP[source_qubit]
                    self._STATE_CACHE_HASH_MAP[src_hashval]= nkp_state['qubit-pair'][0]
                    tgt_hashval = self._LINE_INDEX_MAP[target_qubit]
                    self._LINE_INDEX_MAP[tgt_hashval] = nkp_state['qubit-pair'][1]

        else:
            raise Exception("Exception TBA")
                                    